# RusTorch CoreML Integration - Rust Kernel

このノートブックは、RusTorchでCoreMLを使用する方法を示します。
これはRustカーネルで実行されます。

## 必要な依存関係とfeatureを確認

In [ ]:
// 基本的なRusTorchの使用法
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("RusTorch version: {}", env!("CARGO_PKG_VERSION"));
println!("Rust version: {}", env!("RUSTC_VERSION"));

## CoreMLの可用性をチェック

システムでCoreMLが利用可能かどうかを確認します。

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML available: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreMLが利用可能です！");
        println!("プラットフォーム: macOS");
        
        // デバイス情報を表示
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Cache stats: {:?}", stats);
    } else {
        println!("⚠️ CoreMLは利用できません");
        println!("CPUまたは他のGPUバックエンドを使用してください");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ CoreMLフィーチャーが有効になっていません");
    println!("--features coreml でビルドしてください");
}

## 基本的なテンソル操作

In [ ]:
// 基本的なテンソルを作成
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Tensor A shape: {:?}", a.shape());
println!("Tensor B shape: {:?}", b.shape());

// CPU上で行列乗算
match a.matmul(&b) {
    Ok(result) => {
        println!("行列乗算結果 shape: {:?}", result.shape());
        println!("CPU演算完了 ✓");
    }
    Err(e) => println!("エラー: {:?}", e),
}

## CoreMLバックエンドの使用（利用可能な場合）

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::backend::{CoreMLBackend, CoreMLBackendConfig};
    use rustorch::backends::DeviceManager;
    
    if DeviceManager::is_coreml_available() {
        println!("🚀 CoreMLバックエンドを初期化中...");
        
        let config = CoreMLBackendConfig {
            enable_caching: true,
            max_cache_size: 100,
            auto_fallback: true,
            enable_profiling: true,
        };
        
        match CoreMLBackend::new(config) {
            Ok(backend) => {
                println!("✅ CoreMLバックエンド初期化完了");
                println!("Ready: {}", backend.is_available());
                
                // バックエンド統計を表示
                let stats = backend.get_stats();
                println!("Backend stats: total_operations={}, cache_hits={}, cache_misses={}", 
                         stats.total_operations, stats.cache_hits, stats.cache_misses);
            }
            Err(e) => {
                println!("❌ CoreMLバックエンド初期化失敗: {:?}", e);
            }
        }
    } else {
        println!("⚠️ CoreMLは利用できないため、CPUで実行");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("CoreMLフィーチャーが無効です。CPUで実行します。");
}

## デバイス選択のデモンストレーション

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::smart_device_selector::{
        SmartDeviceSelector, OperationProfile, OperationType
    };
    use rustorch::gpu::DeviceType;
    
    // 利用可能なデバイスを定義
    let available_devices = vec![
        DeviceType::CoreML(0),
        DeviceType::Metal(0),
        DeviceType::Cpu,
    ];
    
    let selector = SmartDeviceSelector::new(available_devices);
    
    // 異なる操作プロファイルでデバイス選択をテスト
    let operations = vec![
        ("小さい行列乗算", OperationProfile::new(
            OperationType::MatrixMultiplication,
            &[16, 16],
            4
        )),
        ("大きい行列乗算", OperationProfile::new(
            OperationType::MatrixMultiplication,
            &[512, 512],
            4
        )),
        ("活性化関数", OperationProfile::new(
            OperationType::Activation,
            &[32, 64, 128, 128],
            4
        )),
        ("複素数演算 (CoreML非対応)", OperationProfile::new(
            OperationType::ComplexNumber,
            &[128, 128],
            8
        )),
    ];
    
    println!("🎯 スマートデバイス選択のデモ:");
    for (name, profile) in operations {
        let selected_device = selector.select_device(&profile);
        println!("  {}: {:?}", name, selected_device);
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("デバイス選択機能はCoreMLフィーチャーが必要です。");
}

## まとめ

このノートブックでは以下を実証しました：

1. ✅ **CoreML可用性チェック**: システムでCoreMLが利用可能かどうかの確認
2. ✅ **基本的なテンソル操作**: CPUでの行列乗算
3. ✅ **CoreMLバックエンド**: 利用可能な場合のCoreMLバックエンド初期化
4. ✅ **スマートデバイス選択**: 操作特性に基づく最適なデバイス選択

### 次のステップ

- より複雑なニューラルネットワーク操作の実装
- パフォーマンス比較（CPU vs CoreML vs Metal）
- 実際のCoreMLモデルとの統合

### 注意事項

- CoreMLはmacOSでのみ利用可能です
- 一部の演算はCoreMLでサポートされていないため、自動的にCPUまたは他のGPUにフォールバックされます
- パフォーマンスは具体的なハードウェアと操作の種類によって異なります